In [1]:
import parse_api
import boto3
import parse_dataset
import os
import pandas as pd
import utils
import warnings
from config import FTP_SERVER_LINK, API_LINK, S3_BUCKET_NAME, INPUT_PATH, API_PATH

utils.clean_temp_files()
pd.set_option('display.float_format', lambda x: '%.2f' % x)
warnings.simplefilter(action='ignore', category=FutureWarning)

## Part 1: AWS S3 & Sourcing Datasets

In [2]:
s3_resource = boto3.resource('s3')

In [3]:
parse_dataset.get_updates(FTP_SERVER_LINK, s3_resource, S3_BUCKET_NAME)

## Part 2: APIs

In [4]:
parse_api.store_api_response(API_LINK, s3_resource, S3_BUCKET_NAME)

In [5]:
utils.sync_s3(s3_resource, S3_BUCKET_NAME)

## Part 3: Data Analytics


### Load both the csv file from Part 1 pr.data.0.Current and the json file from Part 2 as dataframes (Spark, Pyspark, Pandas, Koalas, etc).

In [6]:
#Reading the csv file from Part 1
part1_file = os.path.join(INPUT_PATH, "pr.data.0.Current")
df_part1 = pd.read_csv(part1_file, delimiter = '\t')

#Removing excessive whitespaces from values and column names
df_part1 = df_part1.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df_part1.columns = df_part1.columns.str.replace(' ', '') 

display(df_part1)
df_part1.describe()
df_part1.dtypes

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.60,NaN
1,PRS30006011,1995,Q02,2.10,NaN
2,PRS30006011,1995,Q03,0.90,NaN
3,PRS30006011,1995,Q04,0.10,NaN
4,PRS30006011,1995,Q05,1.40,NaN
...,...,...,...,...,...
35461,PRS88003203,2022,Q04,115.04,NaN
35462,PRS88003203,2022,Q05,113.71,NaN
35463,PRS88003203,2023,Q01,116.21,NaN
35464,PRS88003203,2023,Q02,116.95,NaN


series_id          object
year                int64
period             object
value             float64
footnote_codes     object
dtype: object

In [7]:
#Reading the json file from Part 2
part2_file = os.path.join(API_PATH, "api_response.json")
df_part2 = pd.read_json(part2_file)

#Removing excessive whitespaces from values and column names
df_part2 = df_part2.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df_part2.columns = df_part2.columns.str.replace(' ', '') 

display(df_part2)
df_part2.describe()
df_part2.dtypes


,IDNation,Nation,IDYear,Year,Population,SlugNation
0,01000US,United States,2021,2021,329725481,united-states
1,01000US,United States,2020,2020,326569308,united-states
2,01000US,United States,2019,2019,324697795,united-states
3,01000US,United States,2018,2018,322903030,united-states
4,01000US,United States,2017,2017,321004407,united-states
5,01000US,United States,2016,2016,318558162,united-states
6,01000US,United States,2015,2015,316515021,united-states
7,01000US,United States,2014,2014,314107084,united-states
8,01000US,United States,2013,2013,311536594,united-states


IDNation      object
Nation        object
IDYear         int64
Year           int64
Population     int64
SlugNation    object
dtype: object

### 3.1 Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive.

In [8]:
df2_calculation = df_part2[df_part2["IDYear"].between(2013, 2018)]
display(df2_calculation)

mean_population = df2_calculation['Population'].mean()
std_population = df2_calculation['Population'].std()

print(f"The mean is {mean_population}")
print(f"The standard deviation is {std_population}")

,IDNation,Nation,IDYear,Year,Population,SlugNation
3,01000US,United States,2018,2018,322903030,united-states
4,01000US,United States,2017,2017,321004407,united-states
5,01000US,United States,2016,2016,318558162,united-states
6,01000US,United States,2015,2015,316515021,united-states
7,01000US,United States,2014,2014,314107084,united-states
8,01000US,United States,2013,2013,311536594,united-states


The mean is 317437383.0
The standard deviation is 4257089.5415293295


### 3.2 Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year. For example, if the table had the following values:

In [9]:
df = df_part1.groupby(['series_id', 'year'])['value'].sum()
df = df.reset_index()

max_value_indices = df.groupby('series_id')['value'].transform(max) == df['value']
df_result = df[max_value_indices].reset_index(drop=True)

print("The best year for every series id:")
display(df_result)


The best year for every series id:


,series_id,year,value
0,PRS30006011,2022,20.50
1,PRS30006012,2022,17.10
2,PRS30006013,1998,704.12
3,PRS30006021,2010,17.60
4,PRS30006022,2010,12.50
...,...,...,...
278,PRS88003192,2002,282.80
279,PRS88003193,2022,765.82
280,PRS88003201,2022,36.10
281,PRS88003202,2022,28.90


### 3.3 Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset)

In [10]:
df_1 = df_part1[['series_id', 'year', 'period', 'value']]
df_2 = df_part2[['Year', 'Population']]
df_2.columns = ['year','Population']

df_report = pd.merge(df_1, df_2, on ='year', how='inner')
df_report = df_report[(df_report['series_id'] == "PRS30006032") & (df_report['period'] == "Q01")]
df_report = df_report.reset_index(drop=True)
# df_report["Population"] = df_report["Population"].astype(int) 

print("The report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year")
display(df_report)

The report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year


,series_id,year,period,value,Population
0,PRS30006032,2013,Q01,0.80,311536594
1,PRS30006032,2014,Q01,-0.10,314107084
2,PRS30006032,2015,Q01,-1.60,316515021
3,PRS30006032,2016,Q01,-1.40,318558162
4,PRS30006032,2017,Q01,0.70,321004407
5,PRS30006032,2018,Q01,0.40,322903030
6,PRS30006032,2019,Q01,-2.30,324697795
7,PRS30006032,2020,Q01,-7.00,326569308
8,PRS30006032,2021,Q01,1.30,329725481
